In [1]:
#56_Dhruv_Rajendra_Sawarkar_IITDM
import yaml
from datetime import datetime
import time
import threading
import os

In [2]:
yaml_data = {}
example_fileName = "Milestone1/Milestone1_Example.yaml"
fileA = "Milestone1/Milestone1A.yaml"
fileB = "Milestone1/Milestone1B.yaml"
logFileName = "Milestone1/MileStone1A_Log.txt"
example = 0
filename = fileA
if example == 1:
    filename = example_fileName
with open(filename) as file:
    yaml_data = yaml.safe_load(file)

In [3]:
yaml_data

{'M1A_Workflow': {'Type': 'Flow',
  'Execution': 'Sequential',
  'Activities': {'TaskA': {'Type': 'Task',
    'Function': 'TimeFunction',
    'Inputs': {'FunctionInput': 'TaskA_Input', 'ExecutionTime': '1'}},
   'TaskB': {'Type': 'Task',
    'Function': 'TimeFunction',
    'Inputs': {'FunctionInput': 'TaskB_Input', 'ExecutionTime': '2'}},
   'FlowA': {'Type': 'Flow',
    'Execution': 'Sequential',
    'Activities': {'TaskC': {'Type': 'Task',
      'Function': 'TimeFunction',
      'Inputs': {'FunctionInput': 'TaskC_Input', 'ExecutionTime': '3'}},
     'TaskD': {'Type': 'Task',
      'Function': 'TimeFunction',
      'Inputs': {'FunctionInput': 'TaskD_Input', 'ExecutionTime': '4'}}}}}}}

In [4]:
all_data = {}
global root_logs
root_logs = ""
global history_done
history_done = {}

In [5]:
#creating log file
with open(logFileName,"w") as file:
    file.write("")
def writeLogInFile(log):
    with open(logFileName,"a") as file:
        file.write(log)


In [6]:
def addLog(name,status,optional=""):
    #date time;name status optional
    log = ""
    now = datetime.now() #2022-03-07 10:00:07.000000;
    neededFormat = now.strftime("%Y-%m-%d %H:%M:%S.%f;")
    log += neededFormat
    log += name
    log += status
    log += optional
    log += "\n"
#     root_logs += log
    writeLogInFile(log)
#     print(log)

In [7]:
def doTask(name,task):
    if task['Function'] == "TimeFunction":
        extra = "TimeFunction ("
        values_list = list(task['Inputs'].values())
        n = len(values_list)
        for i in range(n):
            extra += values_list[i]
            if i != n-1:
                extra += ","
        extra += ')'
        addLog(name," Executing ",extra)
        time.sleep(int(task['Inputs']['ExecutionTime']))
        
        

In [8]:
def doTaskSerial(name,task):
    if task['Type'] == 'Flow':
        addLog(name," Entry")
        #do it
        if task['Execution'] == 'Sequential':
            for subtaskName,subtask in task['Activities'].items():
                doTaskSerial(name+"."+subtaskName,subtask)
        elif task['Execution'] == 'Concurrent':
            # do parallel task
            subtasks = []
            threadList = []
            for subtaskName,subtask in task['Activities'].items():
                t = threading.Thread(target = doTaskSerial,args=(name+"."+subtaskName,subtask))
                threadList.append(t)
            for t in threadList:
                t.start()
            for t in threadList:
                t.join()
        else:
            print("\n\n______not serial nor paralle __________\n\n")
        addLog(name," Exit")
    elif task['Type'] == 'Task':
        # no need to go down just do it
        addLog(name," Entry")
        history_done[name] = "notDone"
        doTask(name,task)
        addLog(name," Exit")
        history_done[name] = "done"
            
            
        

In [9]:
for k,v in yaml_data.items():
    doTaskSerial(k,v)

In [10]:
with open(logFileName) as file:
    for line in file:
        print(line)

2022-03-08 11:34:02.524486;M1A_Workflow Entry

2022-03-08 11:34:02.525860;M1A_Workflow.TaskA Entry

2022-03-08 11:34:02.526098;M1A_Workflow.TaskA Executing TimeFunction (TaskA_Input,1)

2022-03-08 11:34:03.527296;M1A_Workflow.TaskA Exit

2022-03-08 11:34:03.527501;M1A_Workflow.TaskB Entry

2022-03-08 11:34:03.527616;M1A_Workflow.TaskB Executing TimeFunction (TaskB_Input,2)

2022-03-08 11:34:05.529771;M1A_Workflow.TaskB Exit

2022-03-08 11:34:05.530158;M1A_Workflow.FlowA Entry

2022-03-08 11:34:05.530344;M1A_Workflow.FlowA.TaskC Entry

2022-03-08 11:34:05.530545;M1A_Workflow.FlowA.TaskC Executing TimeFunction (TaskC_Input,3)

2022-03-08 11:34:08.533020;M1A_Workflow.FlowA.TaskC Exit

2022-03-08 11:34:08.533257;M1A_Workflow.FlowA.TaskD Entry

2022-03-08 11:34:08.533394;M1A_Workflow.FlowA.TaskD Executing TimeFunction (TaskD_Input,4)

2022-03-08 11:34:12.533982;M1A_Workflow.FlowA.TaskD Exit

2022-03-08 11:34:12.534179;M1A_Workflow.FlowA Exit

2022-03-08 11:34:12.534307;M1A_Workflow Exit

